<a href="https://colab.research.google.com/github/rai-sandeep/ai-colab-notebooks/blob/main/Training-V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install

In [17]:
# https://stackoverflow.com/questions/56081324/why-are-google-colab-shell-commands-not-working
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

model_id = "vilsonrodrigues/falcon-7b-instruct-sharded"
trained_model_id = "falcon-7b-instruct-sharded-trained-v1"
ds_id = "rai-sandeep/dataset_full_v1"
example_ques = "Generate a White Paper abstract on Modernizing Enterprise Systems."
example_ques2 = "Generate a White Paper abstract on Bitcoin Mining."
max_tok = 200

save_trained_model=False
%env HF_TOKEN=<ENTER-HUGGINGFACE-TOKEN-HERE>

env: HF_TOKEN=<ENTER-HUGGINGFACE-TOKEN-HERE>


In [2]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U einops
!pip install -q -U safetensors
!pip install -q -U torch
!pip install -q -U xformers
!pip install -q -U datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.1/97.1 MB 8.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.5/268.5 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 60.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 19.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Import

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
import transformers
import torch
from torch.utils.data import DataLoader, Dataset
import torch
from transformers import AutoTokenizer


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('8013'), PosixPath('http'), PosixPath('//172.28.0.1')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//colab.research.google.com/tun/m/cc483011

# Load Quantized Model

In [4]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0}, trust_remote_code=True)

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- configuration_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- modelling_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

In [5]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [6]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [7]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 4718592 || all params: 3613463424 || trainable%: 0.13058363808693696


# Prepare Data

In [8]:
from datasets import load_dataset

data = load_dataset(ds_id)

ds_len=len(data['train'])
rows_sel=data['train'].select(range(1))
rows_sel['format']

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/10 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/rai-sandeep___parquet/rai-sandeep--dataset_full_v1-121f12e8ca170e18/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

['As digital technologies, smart wearables and remote monitoring capabilities penetrate healthcare, traditional healthcare companies are unable to keep up with end-user expectations. Under pressure to adopt rapid transformation, these organizations are looking for robust and end-to-end testing procedures. This paper explains various end-to-end testing approaches within the four main modernization techniques for healthcare companies. The analysis presented here acts as a guideline for healthcare leaders to make strategic and informed decisions on how to modernize their systems based on the needs of their end-users.']

In [9]:
tokenizer.pad_token = tokenizer.eos_token

# prompt_template = "### Instruction: {prompt}\n### Response:"

train_dataset = data['train'].select(range(ds_len)).map(lambda x: {"input_text": x['task']  + "\n" + x['format']})

# Tokenize the datasets
train_encodings = tokenizer(train_dataset['input_text'], max_length=256, truncation=True, padding=True, return_tensors='pt')

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [10]:
  class TextDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = item["input_ids"].clone()
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])


In [14]:
# Convert the encodings to PyTorch datasets
train_dataset = TextDataset(train_encodings)

# Example Before Fine Tuning

In [18]:
def generate(index):

  print("Question:")
  print(example_ques)

  encoding = tokenizer(example_ques, return_tensors="pt").to("cuda:0")
  output = model.generate(input_ids=encoding.input_ids, attention_mask=encoding.attention_mask, max_new_tokens=max_tok, do_sample=True, temperature=0.000001, eos_token_id=tokenizer.eos_token_id, top_k = 0)

  print("Answer:")
  print(tokenizer.decode(output[0], skip_special_tokens=True))

  print("Question2:")
  print(example_ques2)

  encoding = tokenizer(example_ques2, return_tensors="pt").to("cuda:0")
  output = model.generate(input_ids=encoding.input_ids, attention_mask=encoding.attention_mask, max_new_tokens=max_tok, do_sample=True, temperature=0.000001, eos_token_id=tokenizer.eos_token_id, top_k = 0)

  print("Answer:")
  print(tokenizer.decode(output[0], skip_special_tokens=True))

  print()

In [19]:
generate(0)

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Question:
Generate a White Paper abstract on Modernizing Enterprise Systems.


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Answer:
Generate a White Paper abstract on Modernizing Enterprise Systems.
In today's fast-paced digital world, businesses need to keep up with the latest technologies to remain competitive. This white paper explores the challenges and opportunities of modernizing enterprise systems, offering strategies and best practices for businesses to consider when embarking on this critical journey.
Question2:
Generate a White Paper abstract on Bitcoin Mining.
Answer:
Generate a White Paper abstract on Bitcoin Mining.
In this white paper, we explore the process of Bitcoin mining, from the fundamentals of blockchain technology to the specific hardware and software requirements. We examine the economic incentives that drive miners to compete for blocks, as well as the various challenges they face in the race to secure the network. Finally, we dive into the various mining algorithms and their specific advantages and disadvantages, providing a comprehensive overview of the current state of Bitcoin mi

# Training

In [20]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=train_dataset,
    # eval_dataset=val_dataset,
    args=transformers.TrainingArguments(
        num_train_epochs=50,
        per_device_train_batch_size=16,
        gradient_accumulation_steps=4,
        warmup_ratio=0.05,
        # max_steps=100,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit",
        lr_scheduler_type='cosine',
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

#import gc
#del train_encodings, data
#gc.collect()
#torch.cuda.empty_cache()

trainer.train()

<ipython-input-10-bd500a371ea7>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1,0.822000
2,0.822000
3,0.818500
4,0.808600
5,0.788400
6,0.764100
7,0.745000
8,0.739900
9,0.739000
10,0.736000


TrainOutput(global_step=50, training_loss=0.5519472992420197, metrics={'train_runtime': 268.1074, 'train_samples_per_second': 1.865, 'train_steps_per_second': 0.186, 'total_flos': 1751901026304000.0, 'train_loss': 0.5519472992420197, 'epoch': 50.0})

# Example After Fine Tuning

In [21]:
model.config.use_cache = True
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): RWForCausalLM(
      (transformer): RWModel(
        (word_embeddings): Embedding(65024, 4544)
        (h): ModuleList(
          (0-31): 32 x DecoderLayer(
            (input_layernorm): LayerNorm((4544,), eps=1e-05, elementwise_affine=True)
            (self_attention): Attention(
              (maybe_rotary): RotaryEmbedding()
              (query_key_value): Linear4bit(
                in_features=4544, out_features=4672, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4544, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4672, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): Parameter

In [22]:
#example_ques = "Generate a White Paper abstract on Smarter Way To Build System Resilience."
#example_ques2 = "Generate a White Paper abstract on Smarter Way To Build System Resilience. Include the following keywords: available, futuristic, evolving."

generate(0)

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Question:
Generate a White Paper abstract on Modernizing Enterprise Systems.


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Answer:
Generate a White Paper abstract on Modernizing Enterprise Systems.
As organizations continue to digitalize their operations, they are facing significant challenges in maintaining the agility and speed of their systems. This paper examines the current state of enterprise systems and provides a roadmap for modernization that can help organizations achieve greater efficiency and faster time-to-market for digital transformation. The paper also explores the key factors driving the digitalization of enterprise systems and the technologies that can be used to accelerate the modernization process. The paper also highlights the benefits of a well-structured modernization approach and the key considerations that should be made while implementing it. Finally, the paper outlines the key steps and best practices that can be used to successfully modernize enterprise systems.
Question2:
Generate a White Paper abstract on Bitcoin Mining.
Answer:
Generate a White Paper abstract on Bitcoin Minin

# Save the Model

In [23]:
if (save_trained_model):
  trainer.save_model(trained_model_id)
  !huggingface-cli login --token $HF_TOKEN
  model.push_to_hub("rai-sandeep/"+trained_model_id, create_pr=1)

In [28]:
def prompt_and_generate():

  doc_type = input("What type of document do you want to generate? (Enter White Paper or Press Release)\n")
  topic = input(f"On what topic do you want to generate the {doc_type}?\n")
  keywords = input(f"Enter any keywords that should be part of the {doc_type} in comma separated format. If none, press Enter.\n")
  content_to_use = input(f"Enter any content that should be used for the {doc_type}. If none, press Enter.\n")

  doc_content=""
  doc_parts=["Abstract"]

  for doc_part in doc_parts:
    prompt = f"Generate a {doc_type} {doc_part} on {topic}."
    if keywords.strip() != "":
      prompt += f"\nInclude the following keywords: {keywords}."
    if content_to_use.strip() != "":
      prompt += f"\nInclude information from the following content: {content_to_use}.\n"

    encoding = tokenizer(prompt, return_tensors="pt").to("cuda:0")
    output = model.generate(input_ids=encoding.input_ids, attention_mask=encoding.attention_mask, max_new_tokens=max_tok, do_sample=True, temperature=0.000001, eos_token_id=tokenizer.eos_token_id, top_k = 0)

    temp_content=tokenizer.decode(output[0], skip_special_tokens=True)
    temp_content=temp_content.replace(prompt, "")
    doc_content+=f"##{doc_part}  \n{temp_content}\n"

  print(doc_content)

In [29]:
prompt_and_generate()

What type of document do you want to generate? (Enter White Paper or Press Release)
White Paper
On what topic do you want to generate the White Paper?
Enterprise Test Automation
Enter any keywords that should be part of the White Paper in comma separated format. If none, press Enter.

Enter any content that should be used for the White Paper. If none, press Enter.



Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


##Abstract  

As digital transformation continues to accelerate, the need for agile and automated testing becomes even more urgent. This paper examines the current state of test automation in the enterprise and provides a comprehensive guide for establishing a effective test automation program. It outlines the key considerations for successful test automation and offers practical advice for organizations looking to build a robust test automation strategy. The paper also provides a detailed review of the key features of different test automation tools and discusses the benefits of a end-to-end testing approach. Finally, the paper offers a detailed analysis of the end-to-end test automation process and provides a step-by-step guide for setting up a comprehensive test automation program.



Before training:

Question:
Generate a White Paper abstract on Modernizing Enterprise Systems.

Answer:
Generate a White Paper abstract on Modernizing Enterprise Systems.
In today's fast-paced digital world, businesses need to keep up with the latest technologies to remain competitive. This white paper explores the challenges and opportunities of modernizing enterprise systems, offering strategies and best practices for businesses to consider when embarking on this critical journey.

Question2:
Generate a White Paper abstract on Bitcoin Mining.

Answer:
Generate a White Paper abstract on Bitcoin Mining.
In this white paper, we explore the process of Bitcoin mining, from the fundamentals of blockchain technology to the specific hardware and software requirements. We examine the economic incentives that drive miners to compete for blocks, as well as the various challenges they face in the race to secure the network. Finally, we dive into the various mining algorithms and their specific advantages and disadvantages, providing a comprehensive overview of the current state of Bitcoin mining.



After training:

Question:
Generate a White Paper abstract on Modernizing Enterprise Systems.

Answer:
Generate a White Paper abstract on Modernizing Enterprise Systems.
As organizations continue to digitalize their operations, they are facing significant challenges in maintaining the agility and speed of their systems. This paper examines the current state of enterprise systems and provides a roadmap for modernization that can help organizations achieve greater efficiency and faster time-to-market for digital transformation. The paper also explores the key factors driving the digitalization of enterprise systems and the technologies that can be used to accelerate the modernization process. The paper also highlights the benefits of a well-structured modernization approach and the key considerations that should be made while implementing it. Finally, the paper outlines the key steps and best practices that can be used to successfully modernize enterprise systems.

Question2:
Generate a White Paper abstract on Bitcoin Mining.

Answer:
Generate a White Paper abstract on Bitcoin Mining.
As the digital currency landscape evolves, the demand for digital assets continues to grow. This has led to the emergence of new use cases for digital assets, such as distributed storage and smart contracts. In this white paper, we explore the potential of Bitcoin mining as a digital asset mining model. We discuss the current state of the Bitcoin mining ecosystem and the challenges it faces. We also provide an overview of the different types of Bitcoin mining algorithms and their respective advantages and disadvantages. Finally, we outline the potential future of Bitcoin mining and the opportunities it can bring to the digital asset ecosystem.



What type of document do you want to generate? (Enter White Paper or Press Release)

White Paper

On what topic do you want to generate the White Paper?

Enterprise Test Automation

Enter any keywords that should be part of the White Paper in comma separated format. If none, press Enter.

Innovative

Enter any content that should be used for the White Paper. If none, press Enter.

If one word defines the 21st century, it would be speed. Never before has progress happened at such a breakneck pace. We have moved from discrete jumps of innovation to continuous improvement and versioning. The cycle time to production is at an all-time low.

Document Content:
Generate a White Paper abstract on Enterprise Test Automation. Include the following keywords: Innovative. Include information from the following content: If one word defines the 21st century, it would be speed. Never before has progress happened at such a breakneck pace. We have moved from discrete jumps of innovation to continuous improvement and versioning. The cycle time to production is at an all-time low..
Download the White Paper to understand why traditional test automation no longer meets the speed of change and why enterprises need to adopt new approaches to test their applications. The paper outlines the key trends in test automation and provides a comprehensive guide on how to set up a high-performing test automation program. It also offers a deep-dive into the benefits of using continuous testing and its impact on the software development lifecycle. The paper also includes a case study of a successful test automation implementation.


What type of document do you want to generate? (Enter White Paper or Press Release)

White Paper

On what topic do you want to generate the White Paper?

Enterprise Test Automation

Enter any keywords that should be part of the White Paper in comma separated format. If none, press Enter.

Enter any content that should be used for the White Paper. If none, press Enter.


Document Content:
Generate a White Paper abstract on Enterprise Test Automation.
As digital transformation continues to accelerate, the need for agile and automated testing becomes even more urgent. This paper examines the current state of test automation in the enterprise and provides a comprehensive guide on how to establish a successful test automation program. It outlines the key factors to consider when setting up a test automation strategy, including selecting the right toolset, creating a governance model, and establishing a test-driven development pipeline. The paper also offers a detailed look at the benefits of test automation and the key considerations for selecting a test automation tool. Finally, it provides a step-by-step approach for setting up a test automation program that can help organizations achieve greater test coverage, faster test turnaround times, and higher test quality.


What type of document do you want to generate? (Enter White Paper or Press Release)
White Paper
On what topic do you want to generate the White Paper?
Enterprise Test Automation
Enter any keywords that should be part of the White Paper in comma separated format. If none, press Enter.

Enter any content that should be used for the White Paper. If none, press Enter.

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
##Abstract  

As digital transformation continues to accelerate, the need for agile and automated testing becomes even more urgent. This paper examines the current state of test automation in the enterprise and provides a comprehensive guide for establishing a effective test automation program. It outlines the key considerations for successful test automation and offers practical advice for organizations looking to build a robust test automation strategy. The paper also provides a detailed review of the key features of different test automation tools and discusses the benefits of a end-to-end testing approach. Finally, the paper offers a detailed analysis of the end-to-end test automation process and provides a step-by-step guide for setting up a comprehensive test automation program.
